In [ ]:
import nidaqmx 
import numpy as np
import matplotlib.pyplot as plt
import visa
import time
from lantz import MessageBasedDriver, Q_
from lantz.core import Feat
from lantz.core import mfeats
from lantz import ureg

In [ ]:
def adquisicion(task,time,freq):
    dt = np.divide(1,freq)
   # print(dt)
    time_vector = np.arange(0,time,dt)
   # print(len(time_vector))
    num_of_points = len(time_vector)  
    data = task.read(number_of_samples_per_channel=num_of_points)
   # print(len(data))
    return (time_vector,np.asarray(data))

In [ ]:
class Generador(MessageBasedDriver):

    set_query = MessageBasedDriver.write

    @Feat()
    def idn(self):
        return self.query('*IDN?')
    
    frequency = mfeats.QuantityFeat('SOURce1:FREQuency:FIXed?','SOURce1:FREQuency:FIXed {}',units='Hz',limits=(0.0,1000000))
    amplitude = mfeats.QuantityFeat('SOURce1:VOLT:LEV:IMM:AMPL?','SOURce1:VOLT:LEV:IMM:AMPL {}',units='V',limits=(0.05,5))
        
    def setWaveform(self,waveform ='Senoidal'):
        switcher = {'Senoidal':"SIN",'Cuadrada':"SQU",'Pulso':"PULS"}
        self.write("SOURce1:FUNCtion "+switcher.get(waveform,'Senoidal'))  

In [ ]:
def sweepe(gener,init_freq = 100, end_freq = 10100, cant_med = 100):
    with gener as generador:
        paso = np.floor(np.divide(end_freq - init_freq,cant_med))
        values = np.zeros(cant_med)
        freqs = np.add(np.multiply(np.arange(cant_med), paso), init_freq)
        for i,freq in enumerate(freqs):
            periodo = np.divide(1,freq)
        # Escribir Generador
            generador.frequency= freq*ureg.hertz
            print(freq)
        # Esperamos a que se setee y lea bien
            time.sleep(1)

            time_med = .5
            samplefreq = 5000
            nsamples = 5000
            with nidaqmx.Task() as task:
                task.ai_channels.add_ai_voltage_chan('Dev1/ai0',
                                                     terminal_config=nidaqmx.constants.TerminalConfiguration.DIFFERENTIAL)
                task.timing.cfg_samp_clk_timing(samplefreq,samps_per_chan=nsamples)
                (tiempos,data) = adquisicion(task,time_med,samplefreq)

            espectro = plt.magnitude_spectrum(data, Fs=samplefreq, window=np.ones(data.shape))
            max_index = espectro[0].argmax()
            max_freq = espectro[1][max_index]
            
            values[i] = max_freq
    return freqs, values